<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/proteomics_aiagent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mistralai -q
!pip install colab-env -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.8/381.8 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Imports (remain similar)
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed (from original, adapted for quiet install)
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install")
    exit()

# Ensure MISTRAL_API_KEY is set up (from original)
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# --- Pydantic model for Proteomics Analysis Result's response format ---
class ProteinAnalysisResult(BaseModel):
    experiment_id: str
    identified_proteins_count: int
    quantification_summary: dict
    potential_biomarkers: list
    flagged_issues: str
    recommendations: str

print("Creating AI agents for Proteomics Domain...")

# --- Agent Definitions (Proteomics Domain) ---

# 1. Sample Preparation Agent (Refactored from Compound Synthesis Agent)
sample_preparation_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for planning and tracking proteomic sample preparation steps.",
    name="sample-preparation-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "prepare_sample",
                "description": "Simulate the preparation of a biological sample for proteomics (e.g., cell lysis, protein extraction, digestion).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sample_id": {"type": "string", "description": "Identifier for the sample."},
                        "sample_type": {"type": "string", "description": "Type of sample (e.g., 'tissue', 'cell lysate', 'plasma')."},
                        "protocol_id": {"type": "string", "description": "Identifier for the preparation protocol used."},
                    },
                    "required": ["sample_id", "sample_type", "protocol_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "track_sample_status",
                "description": "Log the current status or outcome of a proteomic sample preparation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sample_id": {"type": "string", "description": "Identifier for the sample."},
                        "stage": {"type": "string", "description": "Current stage of preparation (e.g., 'digestion', 'desalting', 'ready for MS')."},
                        "notes": {"type": "string", "description": "Optional: Any relevant notes or observations."},
                    },
                    "required": ["sample_id", "stage"]
                }
            }
        }
    ]
)
print(f"Sample Preparation Agent '{sample_preparation_agent.name}' created with ID: {sample_preparation_agent.id}")

# 2. Mass Spectrometry Data Acquisition Agent (Refactored from Target Binding Agent)
ms_data_acquisition_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for simulating mass spectrometry data acquisition, managing instrument parameters, and checking instrument status.",
    name="ms-data-acquisition-agent",
    tools=[
        {"type": "web_search"}, # For searching external databases for instrument specifications or best practices
        {
            "type": "function",
            "function": {
                "name": "acquire_ms_data",
                "description": "Simulate the acquisition of mass spectrometry data for a prepared sample.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sample_id": {"type": "string", "description": "Identifier of the prepared sample."},
                        "instrument_id": {"type": "string", "description": "Identifier of the mass spectrometer used."},
                        "method_id": {"type": "string", "description": "Identifier for the MS acquisition method."},
                    },
                    "required": ["sample_id", "instrument_id", "method_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_instrument_status",
                "description": "Check the operational status and availability of a specific mass spectrometry instrument.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "instrument_id": {"type": "string", "description": "Identifier of the mass spectrometer."},
                    },
                    "required": ["instrument_id"]
                }
            }
        }
    ]
)
print(f"Mass Spectrometry Data Acquisition Agent '{ms_data_acquisition_agent.name}' created with ID: {ms_data_acquisition_agent.id}")

# 3. Protein Identification and Quantification Agent (Refactored from Drug Efficacy Prediction Agent)
protein_id_quant_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="protein-id-quant-agent",
    description="Agent for identifying proteins from raw MS data, quantifying their abundance, and predicting potential issues.",
    instructions="Identify and quantify proteins from provided mass spectrometry data, provide confidence scores, and flag any ambiguities.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "protein_analysis_result",
                "schema": ProteinAnalysisResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "identify_proteins",
                "description": "Identify proteins from mass spectrometry raw data using a protein database.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "raw_data_file": {"type": "string", "description": "Path or ID of the raw MS data file."},
                        "database_name": {"type": "string", "description": "Name of the protein database (e.g., 'UniProt', 'RefSeq')."},
                    },
                    "required": ["raw_data_file", "database_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "quantify_proteins",
                "description": "Quantify the abundance of identified proteins across different samples or conditions.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "protein_list_json": {"type": "string", "description": "JSON string of identified proteins and their associated data."},
                        "quantification_method": {"type": "string", "description": "Method of quantification (e.g., 'label-free', 'TMT', 'iTRAQ')."},
                    },
                    "required": ["protein_list_json", "quantification_method"]
                }
            }
        }
    ]
)
print(f"Protein Identification and Quantification Agent '{protein_id_quant_agent.name}' created with ID: {protein_id_quant_agent.id}")

# 4. PTM Analysis Agent (Refactored from Clinical Trial Planning Agent)
ptm_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for identifying and analyzing post-translational modifications (PTMs) on proteins, and predicting their functional impact.",
    name="ptm-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "detect_ptms",
                "description": "Detect specific post-translational modifications (e.g., phosphorylation, ubiquitination) from proteomics data.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "protein_id": {"type": "string", "description": "Identifier of the protein to analyze."},
                        "data_type": {"type": "string", "description": "Type of data for PTM detection (e.g., 'MS/MS spectra')."},
                        "ptm_type": {"type": "string", "description": "Optional: Specific PTM type to search for."},
                    },
                    "required": ["protein_id", "data_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_ptm_impact",
                "description": "Predict the functional impact of identified PTMs on protein activity or localization.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "protein_id": {"type": "string", "description": "Identifier of the protein with PTMs."},
                        "ptm_details_json": {"type": "string", "description": "JSON string describing the PTMs (e.g., site, type)."},
                    },
                    "required": ["protein_id", "ptm_details_json"]
                }
            }
        }
    ]
)
print(f"PTM Analysis Agent '{ptm_analysis_agent.name}' created with ID: {ptm_analysis_agent.id}")

# 5. Proteomics Data Repository Agent (Refactored from Compound Library Agent)
proteomics_data_repo_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing and searching proteomics data in a central repository, including raw files, processed data, and metadata.",
    name="proteomics-data-repo-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_data_repository",
                "description": "Search the proteomics data repository for experiments or datasets matching specific criteria.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_criteria": {"type": "string", "description": "Criteria for searching (e.g., 'experiment type', 'sample source', 'date range')."},
                    },
                    "required": ["search_criteria"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "upload_data_to_repository",
                "description": "Simulate uploading processed proteomics data and metadata to the repository.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "experiment_id": {"type": "string", "description": "Identifier of the experiment."},
                        "data_summary_json": {"type": "string", "description": "JSON string summarizing the data to upload."},
                    },
                    "required": ["experiment_id", "data_summary_json"]
                }
            }
        }
    ]
)
print(f"Proteomics Data Repository Agent '{proteomics_data_repo_agent.name}' created with ID: {proteomics_data_repo_agent.id}")

# 6. Bioinformatics Analysis Agent (Refactored from Research Data Analysis Agent)
bioinformatics_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for performing statistical analysis, pathway enrichment, and data visualization on proteomics datasets.",
    name="bioinformatics-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "perform_statistical_analysis",
                "description": "Perform statistical analysis on quantitative proteomics data (e.g., differential expression, ANOVA).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_json": {"type": "string", "description": "JSON string of quantitative protein data."},
                        "analysis_type": {"type": "string", "description": "Type of statistical analysis (e.g., 't-test', 'ANOVA')."},
                        "grouping_info_json": {"type": "string", "description": "JSON string with information for grouping samples."},
                    },
                    "required": ["data_json", "analysis_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "perform_pathway_enrichment",
                "description": "Identify enriched biological pathways or GO terms from a list of differentially expressed proteins.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "protein_list": {"type": "array", "items": {"type": "string"}, "description": "List of protein identifiers."},
                        "organism": {"type": "string", "description": "Organism for pathway analysis (e.g., 'Human', 'Mouse')."},
                    },
                    "required": ["protein_list", "organism"]
                }
            }
        }
    ]
)
print(f"Bioinformatics Analysis Agent '{bioinformatics_analysis_agent.name}' created with ID: {bioinformatics_analysis_agent.id}")

# 7. Quality Control Agent (Refactored from Regulatory Compliance Agent)
qc_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking the quality of raw and processed proteomics data against predefined metrics and identifying potential issues.",
    name="qc-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "run_data_quality_check",
                "description": "Run quality control checks on raw or processed proteomics data.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_id": {"type": "string", "description": "Identifier for the data to check (e.g., raw file, processed matrix)."},
                        "qc_metrics_json": {"type": "string", "description": "JSON string of QC metrics to evaluate."},
                    },
                    "required": ["data_id", "qc_metrics_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "generate_qc_report",
                "description": "Generate a summary report of quality control findings for a proteomics experiment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "experiment_id": {"type": "string", "description": "Identifier of the experiment."},
                        "report_format": {"type": "string", "description": "Desired format for the report (e.g., 'PDF', 'HTML')."},
                    },
                    "required": ["experiment_id"]
                }
            }
        }
    ]
)
print(f"Quality Control Agent '{qc_agent.name}' created with ID: {qc_agent.id}")

# 8. Experimental Design Agent (Refactored from Mechanism of Action Analysis Agent)
experimental_design_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting in the design of proteomics experiments, including sample size calculation and experimental setup.",
    name="experimental-design-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "design_experiment_layout",
                "description": "Design the layout of a proteomics experiment based on research questions and sample types.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "research_question": {"type": "string", "description": "The scientific question the experiment aims to answer."},
                        "sample_types_json": {"type": "string", "description": "JSON string detailing sample types and conditions."},
                        "number_of_replicates": {"type": "number", "description": "Desired number of biological replicates per condition."},
                    },
                    "required": ["research_question", "sample_types_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "calculate_sample_size",
                "description": "Calculate the recommended sample size for a proteomics study based on desired statistical power and effect size.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "desired_power": {"type": "number", "description": "Desired statistical power (e.g., 0.8)."},
                        "effect_size": {"type": "number", "description": "Expected effect size or fold change."},
                        "variability_estimate": {"type": "number", "description": "Estimated variability (e.g., standard deviation from pilot data)."},
                    },
                    "required": ["desired_power", "effect_size", "variability_estimate"]
                }
            }
        }
    ]
)
print(f"Experimental Design Agent '{experimental_design_agent.name}' created with ID: {experimental_design_agent.id}")

# 9. Collaboration & Reporting Agent (Refactored from CRO/Vendor Coordination Agent)
collaboration_reporting_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for coordinating collaborations, managing project communication, and generating final reports for proteomics studies.",
    name="collaboration-reporting-agent",
    tools=[
        {"type": "web_search"}, # For looking up collaborator contact info or reporting guidelines.
        {
            "type": "function",
            "function": {
                "name": "share_results_with_collaborator",
                "description": "Simulate sharing proteomics results or data with a specified collaborator.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "collaborator_id": {"type": "string", "description": "Identifier of the collaborator."},
                        "data_summary": {"type": "string", "description": "Summary of the data or results being shared."},
                        "access_level": {"type": "string", "description": "Level of access granted (e.g., 'view-only', 'edit')."},
                    },
                    "required": ["collaborator_id", "data_summary"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "generate_final_report",
                "description": "Generate a comprehensive final report for a proteomics study.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {"type": "string", "description": "Identifier of the proteomics project."},
                        "report_sections_json": {"type": "string", "description": "JSON string specifying sections to include (e.g., 'methods', 'results', 'conclusions')."},
                    },
                    "required": ["project_id"]
                }
            }
        }
    ]
)
print(f"Collaboration & Reporting Agent '{collaboration_reporting_agent.name}' created with ID: {collaboration_reporting_agent.id}")

print("\nAll new proteomics-related agents have been defined.")

# --- Mock functions for proteomics domain tools ---

def prepare_sample(sample_id: str, sample_type: str, protocol_id: str):
    """MOCK function to prepare a biological sample for proteomics."""
    print(f"\n[DEBUG] MOCK CALL: prepare_sample ID: '{sample_id}', Type: '{sample_type}', Protocol: '{protocol_id}'")
    if "tissue" in sample_type.lower() and "standard" in protocol_id.lower():
        return {"status": "success", "sample_id": sample_id, "preparation_details": f"{sample_type} prepared via {protocol_id}. Protein extraction successful.", "message": "Sample ready for MS."}
    return {"status": "success", "sample_id": sample_id, "preparation_details": f"Sample {sample_id} prepared. Review required.", "message": "Preparation complete with notes."}

def track_sample_status(sample_id: str, stage: str, notes: str = None):
    """MOCK function to log the current status of a proteomic sample preparation."""
    print(f"\n[DEBUG] MOCK CALL: track_sample_status Sample: {sample_id}, Stage: {stage}, Notes: {notes}")
    return {"status": "logged", "sample_id": sample_id, "stage": stage, "timestamp": time.time(), "message": "Sample status updated."}

def acquire_ms_data(sample_id: str, instrument_id: str, method_id: str):
    """MOCK function to acquire mass spectrometry data."""
    print(f"\n[DEBUG] MOCK CALL: acquire_ms_data Sample: '{sample_id}', Instrument: '{instrument_id}', Method: '{method_id}'")
    if "S-001" in sample_id and "Orbitrap01" in instrument_id:
        return {"status": "data_acquired", "raw_file_id": f"{sample_id}_{instrument_id}_{method_id}.raw", "scan_count": 12000, "message": "High-resolution MS data acquired successfully."}
    return {"status": "data_acquired", "raw_file_id": f"{sample_id}_generic.raw", "scan_count": 5000, "message": "MS data acquired."}

def check_instrument_status(instrument_id: str):
    """MOCK function to check the operational status of a mass spectrometer."""
    print(f"\n[DEBUG] MOCK CALL: check_instrument_status Instrument: {instrument_id}")
    if instrument_id == "Orbitrap01":
        return {"status": "online", "last_calibration": "2025-06-28", "uptime_hours": 72, "message": "Instrument is online and calibrated."}
    return {"status": "offline", "message": "Instrument is offline or not found."}

def identify_proteins(raw_data_file: str, database_name: str):
    """MOCK function to identify proteins from mass spectrometry raw data."""
    print(f"\n[DEBUG] MOCK CALL: identify_proteins Raw File: '{raw_data_file}', Database: '{database_name}'")
    if "S-001_Orbitrap01_methodA.raw" in raw_data_file and "UniProt" in database_name:
        return {"status": "success", "identified_proteins": ["P001_HUMAN", "P002_HUMAN", "Q003_MOUSE"], "match_confidence": 0.9, "message": "Proteins identified with high confidence."}
    return {"status": "success", "identified_proteins": ["P999_UNKNOWN"], "match_confidence": 0.4, "message": "Protein identification complete, review needed."}

def quantify_proteins(protein_list_json: str, quantification_method: str):
    """MOCK function to quantify the abundance of identified proteins."""
    print(f"\n[DEBUG] MOCK CALL: quantify_proteins Protein List: {protein_list_json}, Method: {quantification_method}")
    try:
        proteins_data = json.loads(protein_list_json)
        # Simulate some quantification
        quant_results = {}
        for protein in proteins_data:
            quant_results[protein] = {"abundance": round(time.time() % 1000, 2), "fold_change": round((time.time() % 10) + 1, 2)}
        return {"status": "success", "quantification_results": quant_results, "method": quantification_method, "message": "Proteins quantified."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for protein list."}

def detect_ptms(protein_id: str, data_type: str, ptm_type: str = None):
    """MOCK function to detect specific post-translational modifications."""
    print(f"\n[DEBUG] MOCK CALL: detect_ptms Protein: {protein_id}, Data Type: {data_type}, PTM Type: {ptm_type}")
    if protein_id == "P001_HUMAN" and "MS/MS" in data_type:
        if ptm_type and "phosphorylation" in ptm_type.lower():
            return {"status": "ptm_detected", "protein": protein_id, "ptm_sites": ["Ser123", "Thr456"], "ptm_type": "Phosphorylation", "confidence": 0.9}
        return {"status": "ptm_detected", "protein": protein_id, "ptm_sites": ["Lys78"], "ptm_type": "Ubiquitination", "confidence": 0.85}
    return {"status": "no_ptm_detected", "message": "No significant PTMs detected for this protein/data."}

def predict_ptm_impact(protein_id: str, ptm_details_json: str):
    """MOCK function to predict the functional impact of identified PTMs."""
    print(f"\n[DEBUG] MOCK CALL: predict_ptm_impact Protein: {protein_id}, PTM Details: {ptm_details_json}")
    try:
        ptm_details = json.loads(ptm_details_json)
        if "Phosphorylation" in ptm_details.get("ptm_type", ""):
            return {"status": "impact_predicted", "protein": protein_id, "predicted_impact": "Likely alters protein activity or signaling.", "confidence": 0.92}
        return {"status": "impact_predicted", "protein": protein_id, "predicted_impact": "Potential structural or stability change.", "confidence": 0.75}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for PTM details."}

def search_data_repository(search_criteria: str):
    """MOCK function to search the proteomics data repository."""
    print(f"\n[DEBUG] MOCK CALL: search_data_repository for criteria: {search_criteria}")
    if "experiment type: quantitative" in search_criteria.lower():
        return {"status": "success", "datasets_found": ["EXP-001_Quant", "EXP-005_TMT"], "message": "Found quantitative proteomics datasets."}
    return {"status": "success", "datasets_found": [], "message": "No datasets matching criteria."}

def upload_data_to_repository(experiment_id: str, data_summary_json: str):
    """MOCK function to simulate uploading processed proteomics data."""
    print(f"\n[DEBUG] MOCK CALL: upload_data_to_repository Experiment: {experiment_id}, Summary: {data_summary_json}")
    return {"status": "uploaded", "experiment_id": experiment_id, "upload_timestamp": time.time(), "message": "Data uploaded to repository."}

def perform_statistical_analysis(data_json: str, analysis_type: str, grouping_info_json: str = None):
    """MOCK function to perform statistical analysis on proteomics data."""
    print(f"\n[DEBUG] MOCK CALL: perform_statistical_analysis Data: {data_json}, Type: {analysis_type}, Grouping: {grouping_info_json}")
    # Simple mock: if data is provided, assume significance
    if data_json and "differential" in analysis_type.lower():
        return {"status": "success", "analysis_result": "Differentially expressed proteins found.", "p_value_threshold": 0.05, "significant_hits": 15}
    return {"status": "success", "analysis_result": "Analysis performed, no significant changes.", "p_value_threshold": 0.05, "significant_hits": 0}

def perform_pathway_enrichment(protein_list: list, organism: str):
    """MOCK function to identify enriched biological pathways."""
    print(f"\n[DEBUG] MOCK CALL: perform_pathway_enrichment Proteins: {protein_list}, Organism: {organism}")
    if "P001_HUMAN" in protein_list and "Human" in organism:
        return {"status": "success", "enriched_pathways": ["Apoptosis pathway", "Cell Cycle Regulation"], "message": "Pathways enriched."}
    return {"status": "success", "enriched_pathways": [], "message": "No significant pathway enrichment."}

def run_data_quality_check(data_id: str, qc_metrics_json: str):
    """MOCK function to run quality control checks on proteomics data."""
    print(f"\n[DEBUG] MOCK CALL: run_data_quality_check Data: {data_id}, Metrics: {qc_metrics_json}")
    try:
        metrics = json.loads(qc_metrics_json)
        if "TIC" in metrics and metrics["TIC"] > 1e7: # Example QC metric
            return {"status": "passed_qc", "data_id": data_id, "qc_score": 0.95, "flags": [], "message": "Data passed all quality checks."}
        return {"status": "failed_qc", "data_id": data_id, "qc_score": 0.6, "flags": ["Low TIC", "High CV"], "message": "Data failed some QC metrics."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for QC metrics."}

def generate_qc_report(experiment_id: str, report_format: str = "HTML"):
    """MOCK function to generate a summary report of quality control findings."""
    print(f"\n[DEBUG] MOCK CALL: generate_qc_report Experiment: {experiment_id}, Format: {report_format}")
    return {"status": "report_generated", "report_id": f"QC_Report_{experiment_id}", "format": report_format, "message": "QC report successfully generated."}

def design_experiment_layout(research_question: str, sample_types_json: str, number_of_replicates: int = 3):
    """MOCK function to design the layout of a proteomics experiment."""
    print(f"\n[DEBUG] MOCK CALL: design_experiment_layout Question: '{research_question}', Samples: {sample_types_json}, Replicates: {number_of_replicates}")
    try:
        sample_types = json.loads(sample_types_json)
        layout = {"conditions": sample_types, "replicates_per_condition": number_of_replicates, "total_samples": len(sample_types) * number_of_replicates}
        return {"status": "design_complete", "experiment_layout": layout, "message": "Experimental layout designed."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for sample types."}

def calculate_sample_size(desired_power: float, effect_size: float, variability_estimate: float):
    """MOCK function to calculate the recommended sample size for a proteomics study."""
    print(f"\n[DEBUG] MOCK CALL: calculate_sample_size Power: {desired_power}, Effect: {effect_size}, Variability: {variability_estimate}")
    # Simple mock calculation
    required_samples = int((2 * (variability_estimate ** 2) * ((1.96 + 0.84)**2)) / (effect_size ** 2)) # Approximation for power=0.8, alpha=0.05
    return {"status": "calculated", "recommended_sample_size": max(required_samples, 5), "message": "Sample size calculation complete."} # Ensure min 5

def share_results_with_collaborator(collaborator_id: str, data_summary: str, access_level: str = "view-only"):
    """MOCK function to simulate sharing proteomics results with a collaborator."""
    print(f"\n[DEBUG] MOCK CALL: share_results_with_collaborator Collaborator: {collaborator_id}, Summary: '{data_summary}', Access: {access_level}")
    return {"status": "shared", "collaborator": collaborator_id, "message": f"Results shared with {collaborator_id} with {access_level} access."}

def generate_final_report(project_id: str, report_sections_json: str = None):
    """MOCK function to generate a comprehensive final report for a proteomics study."""
    print(f"\n[DEBUG] MOCK CALL: generate_final_report Project: {project_id}, Sections: {report_sections_json}")
    return {"status": "report_generated", "report_file": f"Final_Report_{project_id}.pdf", "message": "Final report generated and ready for review."}

# Master Tool Executor Mapping (Updated for Proteomics Domain)
tool_executor = {
    "prepare_sample": prepare_sample,
    "track_sample_status": track_sample_status,
    "acquire_ms_data": acquire_ms_data,
    "check_instrument_status": check_instrument_status,
    "identify_proteins": identify_proteins,
    "quantify_proteins": quantify_proteins,
    "detect_ptms": detect_ptms,
    "predict_ptm_impact": predict_ptm_impact,
    "search_data_repository": search_data_repository,
    "upload_data_to_repository": upload_data_to_repository,
    "perform_statistical_analysis": perform_statistical_analysis,
    "perform_pathway_enrichment": perform_pathway_enrichment,
    "run_data_quality_check": run_data_quality_check,
    "generate_qc_report": generate_qc_report,
    "design_experiment_layout": design_experiment_layout,
    "calculate_sample_size": calculate_sample_size,
    "share_results_with_collaborator": share_results_with_collaborator,
    "generate_final_report": generate_final_report,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General proteomics information retrieved."
}

# Function to standardize tools for client.chat.complete (remains the same)
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Proteomics Agents ---
print("\n--- Executing Test Cases for Proteomics Agents (via chat completions) ---")
test_cases = [
    {
        "agent": sample_preparation_agent,
        "name": "Sample Preparation Agent",
        "query": "Prepare a plasma sample for proteomics, using the 'Standard_Digest_Protocol_v2' protocol.",
        "expected_tool_call": "prepare_sample"
    },
    {
        "agent": sample_preparation_agent,
        "name": "Sample Preparation Agent",
        "query": "Log the status for sample P-001: currently undergoing desalting.",
        "expected_tool_call": "track_sample_status"
    },
    {
        "agent": ms_data_acquisition_agent,
        "name": "Mass Spectrometry Data Acquisition Agent",
        "query": "Acquire MS data for sample S-001 using Orbitrap01 instrument and 'Method_A_DIA'.",
        "expected_tool_call": "acquire_ms_data"
    },
    {
        "agent": ms_data_acquisition_agent,
        "name": "Mass Spectrometry Data Acquisition Agent",
        "query": "Check the status of Orbitrap01.",
        "expected_tool_call": "check_instrument_status"
    },
    {
        "agent": protein_id_quant_agent,
        "name": "Protein Identification and Quantification Agent",
        "query": "Identify proteins from raw data file 'S-001_Orbitrap01_methodA.raw' using the UniProt database.",
        "expected_tool_call": "identify_proteins"
    },
    {
        "agent": protein_id_quant_agent,
        "name": "Protein Identification and Quantification Agent",
        "query": "Quantify proteins: [\"P001_HUMAN\", \"P002_HUMAN\"] using label-free quantification.",
        "expected_tool_call": "quantify_proteins"
    },
    {
        "agent": ptm_analysis_agent,
        "name": "PTM Analysis Agent",
        "query": "Detect phosphorylation PTMs on protein P001_HUMAN using MS/MS spectra data.",
        "expected_tool_call": "detect_ptms"
    },
    {
        "agent": ptm_analysis_agent,
        "name": "PTM Analysis Agent",
        "query": "Predict impact of PTMs for protein P001_HUMAN: {\"ptm_type\": \"Phosphorylation\", \"site\": \"Ser123\"}.",
        "expected_tool_call": "predict_ptm_impact"
    },
    {
        "agent": proteomics_data_repo_agent,
        "name": "Proteomics Data Repository Agent",
        "query": "Search data repository for quantitative proteomics experiments.",
        "expected_tool_call": "search_data_repository"
    },
    {
        "agent": proteomics_data_repo_agent,
        "name": "Proteomics Data Repository Agent",
        "query": "Upload processed data for experiment EXP-002: {\"proteins\": 1500, \"samples\": 10, \"type\": \"TMT_Quantification\"}.",
        "expected_tool_call": "upload_data_to_repository"
    },
    {
        "agent": bioinformatics_analysis_agent,
        "name": "Bioinformatics Analysis Agent",
        "query": "Perform differential expression analysis on data: [{\"protein\": \"P1\", \"control\": 100, \"treated\": 200}, {\"protein\": \"P2\", \"control\": 50, \"treated\": 60}]",
        "expected_tool_call": "perform_statistical_analysis"
    },
    {
        "agent": bioinformatics_analysis_agent,
        "name": "Bioinformatics Analysis Agent",
        "query": "Perform pathway enrichment for proteins: [\"P001_HUMAN\", \"P002_HUMAN\"] in Human.",
        "expected_tool_call": "perform_pathway_enrichment"
    },
    {
        "agent": qc_agent,
        "name": "Quality Control Agent",
        "query": "Run quality check on raw file 'Sample_X_MS.raw' with metrics: {\"TIC\": 1e8, \"peak_width\": 0.1}.",
        "expected_tool_call": "run_data_quality_check"
    },
    {
        "agent": qc_agent,
        "name": "Quality Control Agent",
        "query": "Generate a QC report for experiment EXP-003.",
        "expected_tool_call": "generate_qc_report"
    },
    {
        "agent": experimental_design_agent,
        "name": "Experimental Design Agent",
        "query": "Design an experiment to study protein changes in cancer vs normal tissue, with 5 replicates per group.",
        "expected_tool_call": "design_experiment_layout"
    },
    {
        "agent": experimental_design_agent,
        "name": "Experimental Design Agent",
        "query": "Calculate sample size for a study with 0.9 power, 2.0 effect size, and 0.5 variability.",
        "expected_tool_call": "calculate_sample_size"
    },
    {
        "agent": collaboration_reporting_agent,
        "name": "Collaboration & Reporting Agent",
        "query": "Share final proteomics results for Project-Bio with Dr. Smith.",
        "expected_tool_call": "share_results_with_collaborator"
    },
    {
        "agent": collaboration_reporting_agent,
        "name": "Collaboration & Reporting Agent",
        "query": "Generate a final report for Project-Bio.",
        "expected_tool_call": "generate_final_report"
    },
    {
        "agent": collaboration_reporting_agent,
        "name": "Collaboration & Reporting Agent (Web Search Example)",
        "query": "Search for best practices in proteomics data reporting guidelines.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control
    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")
    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})
    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        response_turn1 = client.chat.complete(
            messages=conversation_history,
            model=agent_to_test.model,
            tools=api_call_tools_list,
        )
        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")
                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executo"})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call reque
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response f")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the
            )
            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice)
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)
        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)
    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime restart) might be necessary.")

print("\n--- All test cases execution complete. ---")

Mounted at /content/gdrive
Creating AI agents for Proteomics Domain...
Sample Preparation Agent 'sample-preparation-agent' created with ID: ag_0197c719725d727d8c341ca519a2945b
Mass Spectrometry Data Acquisition Agent 'ms-data-acquisition-agent' created with ID: ag_0197c719739971ab84b106c905e11b85
Protein Identification and Quantification Agent 'protein-id-quant-agent' created with ID: ag_0197c71974677760a715cabeb27119e5
PTM Analysis Agent 'ptm-analysis-agent' created with ID: ag_0197c7197529713883bfcdc453b20e71
Proteomics Data Repository Agent 'proteomics-data-repo-agent' created with ID: ag_0197c719760d7359979cd6a233ff83d1
Bioinformatics Analysis Agent 'bioinformatics-analysis-agent' created with ID: ag_0197c71976ec702f8171a2ab986c4d3d
Quality Control Agent 'qc-agent' created with ID: ag_0197c71978f270f1b6d0a186e6dc6167
Experimental Design Agent 'experimental-design-agent' created with ID: ag_0197c71979d577fcaa98a7eb8171344e
Collaboration & Reporting Agent 'collaboration-reporting-age